## Environment setup

In [ ]:
!pip install opendatasets --upgrade --quiet
!pip install scikit-learn --upgrade --quiet
!pip install fairlearn --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import opendatasets as od
import sklearn
from fairlearn.metrics import demographic_parity_difference, demographic_parity_ratio, equalized_odds_difference, equalized_odds_ratio

In [ ]:
#NECESITA UN kaggle.json (API key) que se puede sacar en configuración desde kaggle

## 2.1) Selección del dataset

In [ ]:
dataset_url = "https://www.kaggle.com/datasets/satayjit/student-performance-bd"
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alanc26
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/satayjit/student-performance-bd


100%|██████████| 263k/263k [00:00<00:00, 609kB/s]

In [ ]:
df = pd.read_csv("./student-performance-bd/bd_students_per.csv")
df.sample(10)

,id,full_name,age,gender,location,family_size,mother_education,father_education,mother_job,father_job,...,tutoring,school_type,attendance,extra_curricular_activities,english,math,science,social_science,art_culture,stu_group
7432,7434,Mst. Moniratul Jannati Mou,18,Female,Urban,7,HSC,Hons,No,Yes,...,Yes,Govt,89,Yes,75,74,72,72,80,science
3646,3648,Papia Halder,15,Female,City,2,SSC,SSC,No,Yes,...,No,Govt,62,Yes,78,70,74,72,72,commerce
7918,7920,Vikas Iqbal,15,Male,City,3,Under_SSC,Masters,Yes,Yes,...,Yes,Semi_Govt,97,No,94,85,87,87,90,science
3863,3865,Kazi Miah,17,Male,Rural,4,HSC,Diploma,No,Yes,...,No,Semi_Govt,71,Yes,50,66,68,86,44,arts
4673,4675,Ahsanullah Sen,18,Male,Rural,5,Under_SSC,Diploma,No,Yes,...,No,Semi_Govt,56,No,63,48,64,67,57,arts
5509,5511,Badol Reza,15,Male,Urban,6,Masters,Masters,No,No,...,Yes,Semi_Govt,92,No,70,61,64,50,55,arts
3383,3385,Delowar Bhuiyan,16,Male,City,3,SSC,Non_Educated,Yes,No,...,Yes,Semi_Govt,61,No,76,80,70,81,79,commerce
257,259,Chayon Kumar Halder,16,Male,Urban,4,Hons,HSC,No,Yes,...,No,Private,84,No,78,71,72,74,86,Commerce
1828,1830,Afsari Paul,16,Female,Rural,5,Under_SSC,SSC,Yes,No,...,Yes,Govt,56,Yes,40,57,50,61,63,arts
2120,2122,Tanvir Akter,18,Female,City,2,SSC,Non_Educated,Yes,Yes,...,Yes,Private,74,No,74,78,81,86,92,commerce


### Pre-processing

In [ ]:
import statistics

# Métricas para asignar aprobación/reprobación según resultados en pruebas
def f(english, math,	science,	social_science,	art_culture):
  grades= [english,math,science,social_science,art_culture]
  mean = statistics.mean(grades)
  if mean >= 60:
    return 1
  else:
    return 0

In [ ]:
df['approved'] = df.apply(lambda x: f(x.english, x.math, x.science, x.social_science, x.art_culture), axis=1)
df.drop(columns=['english','math','science', 'social_science', 'art_culture', 'full_name', 'id'], inplace=True)

In [ ]:
df = df.dropna()

In [ ]:
df.sample(5)

,age,gender,location,family_size,mother_education,father_education,mother_job,father_job,guardian,parental_involvement,internet_access,studytime,tutoring,school_type,attendance,extra_curricular_activities,stu_group,approved
8108,17,Female,City,6,HSC,Under_SSC,No,No,Mother,Yes,No,1,Yes,Private,77,Yes,arts,0
2387,16,Male,Rural,2,Honors,Diploma,No,Yes,Mother,Yes,Yes,3,Yes,Private,71,Yes,arts,0
6923,18,Male,Urban,6,Diploma,Under_SSC,No,Yes,Father,Yes,No,2,Yes,Private,84,No,arts,0
7924,15,Female,Rural,5,Non_Educated,Non_Educated,No,No,Other,No,Yes,5,No,Govt,73,Yes,commerce,1
6492,17,Male,City,7,SSC,Masters,No,No,Father,No,No,2,Yes,Govt,73,Yes,arts,0


In [ ]:
# Funciones para asignar valores a datos no numéricos

def multiple_to_int(x, value_list):
  return value_list.index(x)

def multiple_col_to_int_col(df, col=None, cols=None):
  if col:
    multiple_col_to_int_col(df, cols=[col])
  if cols:
    for col in cols:
      value_list = list(df[col].unique())
      df[col] = df.apply(lambda x: multiple_to_int(x[col], value_list), axis=1)
  return df

In [ ]:
df = multiple_col_to_int_col(df, cols=["mother_job", "father_job", "parental_involvement", "internet_access", "tutoring", "extra_curricular_activities",
                                       "mother_education", "father_education", "guardian", "school_type", "stu_group", "location", "gender"])
df.sample(5)

,age,gender,location,family_size,mother_education,father_education,mother_job,father_job,guardian,parental_involvement,internet_access,studytime,tutoring,school_type,attendance,extra_curricular_activities,stu_group,approved
2101,16,0,1,6,1,6,0,0,1,1,0,3,0,2,58,1,5,0
3952,18,1,2,7,3,0,1,1,2,0,1,5,0,2,63,1,4,1
6076,17,1,2,4,4,4,0,0,2,1,1,3,0,1,68,1,5,1
3169,18,0,0,5,7,0,0,1,0,1,1,1,1,1,70,0,5,0
5700,17,1,2,2,0,6,0,0,1,0,0,1,0,0,67,0,5,0


## 2.2) Entrenamiento del modelo

In [ ]:
total_rows= df.shape[0]
pctg_training = 70

train_size = int(pctg_training/100*total_rows)

df = df.sample(frac=1) #shuffle

training = df[:train_size]
testing = df[train_size:]

X_train = training.drop(columns=["approved"])
Y_train = training["approved"]

X_test = testing.drop(columns=["approved"])
Y_test = testing["approved"]

### Decision tree

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, Y_train)

In [ ]:
row = df.sample(1)
expected = int(row["approved"])
predicted = int(clf.predict(row.drop(columns=["approved"])))
print(f"Expected: { expected }")
print(f"Predicted: { predicted }")

Expected: 1
Predicted: 1


<ipython-input-13-eef0485560cf>:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  expected = int(row["approved"])
<ipython-input-13-eef0485560cf>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted = int(clf.predict(row.drop(columns=["approved"])))


In [ ]:
from sklearn.metrics import precision_score, f1_score

# Calculamos la precisión y el f1 score

y_true = Y_test
y_pred_t = clf.predict(X_test)

precision = precision_score(y_true, y_pred_t, average='macro')
f1_score = f1_score(y_true, y_pred_t, average='macro')
print(f"Precision score: {precision} \nF1 score: {f1_score}")

Precision score: 0.8500462439347214 
F1 score: 0.8374024871780166


### Logistic regresion

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, Y_train)

In [ ]:
row = df.sample(1)
expected = int(row["approved"])
predicted = int(clf.predict(row.drop(columns=["approved"])))
print(f"Expected: { expected }")
print(f"Predicted: { predicted }")

Expected: 0
Predicted: 0


<ipython-input-16-eef0485560cf>:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  expected = int(row["approved"])
<ipython-input-16-eef0485560cf>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted = int(clf.predict(row.drop(columns=["approved"])))


In [ ]:
from sklearn.metrics import precision_score, f1_score

# Calculamos la precisión y el f1 score

y_true = Y_test
y_pred_lr = clf.predict(X_test)

precision = precision_score(y_true, y_pred_lr, average='macro')
f1_score = f1_score(y_true, y_pred_lr, average='macro')
print(f"Precision score: {precision} \nF1 score: {f1_score}")

Precision score: 0.8746570091709323 
F1 score: 0.8981256512545421


### Support Vector Machines

In [ ]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train, Y_train)

SVC()

In [ ]:
row = df.sample(1)
expected = int(row["approved"])
predicted = int(clf.predict(row.drop(columns=["approved"])))
print(f"Expected: { expected }")
print(f"Predicted: { predicted }")

Expected: 1
Predicted: 1


<ipython-input-19-eef0485560cf>:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  expected = int(row["approved"])
<ipython-input-19-eef0485560cf>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted = int(clf.predict(row.drop(columns=["approved"])))


In [ ]:
from sklearn.metrics import precision_score, f1_score

# Calculamos la precisión y el f1 score
y_true = Y_test
y_pred_svm = clf.predict(X_test)

precision = precision_score(y_true, y_pred_svm, average='macro')
f1_score = f1_score(y_true, y_pred_svm, average='macro')
print(f"Precision score: {precision} \nF1 score: {f1_score}")

Precision score: 0.8659423686233337 
F1 score: 0.8850747884970257


## 2.3) Medición de sesgo

In [ ]:
y_true = Y_test
y_pred = y_pred_lr
mother_job = X_test["mother_job"]
gender = X_test["gender"]

### Independencia

El atributo sensible es mother_job.

In [ ]:
dp_diff = demographic_parity_difference(y_true, y_pred, sensitive_features= mother_job)
dp_ratio = demographic_parity_ratio(y_true, y_pred, sensitive_features= mother_job)

print(dp_diff)
print(dp_ratio)

0.020667969889290316
0.9712427222237129


El atributo sensible es gender.

In [ ]:
dp_diff = demographic_parity_difference(y_true, y_pred, sensitive_features= gender)
dp_ratio = demographic_parity_ratio(y_true, y_pred, sensitive_features= gender)

print(dp_diff)
print(dp_ratio)

0.00026561925342960535
0.9996253862674404


### Separación

El atributo sensible es mother_job.

In [ ]:
eo_diff = equalized_odds_difference(y_true, y_pred, sensitive_features= mother_job)
eo_ratio = equalized_odds_ratio(y_true, y_pred, sensitive_features= mother_job)

print(eo_diff)
print(eo_ratio)

0.025087783766127747
0.534453781512605


El atributo sensible es gender.

In [ ]:
eo_diff = equalized_odds_difference(y_true, y_pred, sensitive_features= gender)
eo_ratio = equalized_odds_ratio(y_true, y_pred, sensitive_features= gender)

print(eo_diff)
print(eo_ratio)

0.011927745193874295
0.9803713527851459


### Suficiencia

In [ ]:
from sklearn.metrics import precision_score

# Función para calcular la Predictive Parity
def predictive_parity(y_true, y_pred, sensitive_feature):
    groups = sensitive_feature.unique()
    precision_scores = {}
    for group in groups:
        mask = sensitive_feature == group
        precision_scores[group] = precision_score(y_true[mask], y_pred[mask])
    return precision_scores

In [ ]:
pp_mother_job = predictive_parity(y_true, y_pred, mother_job)
print("Predictive Parity for mother_job:")
for job, score in pp_mother_job.items():
    print(f"Group {job}: Precision {score}")

Predictive Parity for mother_job:
Group 0: Precision 0.9825819672131147
Group 1: Precision 0.9906213364595545


In [ ]:
pp_gender = predictive_parity(y_true, y_pred, gender)
print("\nPredictive Parity for gender:")
for gen, score in pp_gender.items():
    print(f"Group {gen}: Precision {score}")


Predictive Parity for gender:
Group 1: Precision 0.9867986798679867
Group 0: Precision 0.9858695652173913


## 2.4) Mitigación de sesgo


In [ ]:
!pip install aif360

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install aif360[inFairness]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.6/865.6 kB 25.3 MB/s eta 0:00:00


### 2.4.1) Pre-procesamiento

In [ ]:
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing

protected_attributes = ['mother_job', 'gender']
unprivileged_groups = [{'mother_job': 0, 'gender': 0}]
privileged_groups = [{'mother_job': 1, 'gender': 1}]

data_aif = BinaryLabelDataset(
    df=pd.concat([X_train, Y_train], axis=1),
    label_names=['approved'],
    protected_attribute_names=protected_attributes
)

# Aplica el Reweighting
rw = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
data_rw = rw.fit_transform(data_aif)

# Entrenamiento del modelo con la data Reweighted
X_train_rw = data_rw.features
y_train_rw = data_rw.labels.ravel()
clf_rw = LogisticRegression(random_state=0, max_iter=1000).fit(X_train_rw, y_train_rw)

In [ ]:
from aif360.metrics import BinaryLabelDatasetMetric

# Métricas del modelo original
metric_orig_train = BinaryLabelDatasetMetric(
    data_aif,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

print("Original training dataset metrics:")
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

# Métricas para el modelo Reweighted
metric_rw_train = BinaryLabelDatasetMetric(
    data_rw,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

print("\nReweighted training dataset metrics:")
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_rw_train.mean_difference())

Original training dataset metrics:
Difference in mean outcomes between unprivileged and privileged groups = -0.002918

Reweighted training dataset metrics:
Difference in mean outcomes between unprivileged and privileged groups = -0.000000


### 2.4.2) In-procesamiento

In [ ]:
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.metrics import BinaryLabelDatasetMetric

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

sess = tf.Session()

# Inicialización del modelo Adversarial Debiasing
adv_debiasing = AdversarialDebiasing(privileged_groups=privileged_groups,
                                     unprivileged_groups=unprivileged_groups,
                                     scope_name='adversarial_debiasing',
                                     debias=True,
                                     seed=66,
                                     sess=sess)

# Entrenamiento del modelo usando el dataset original
adv_debiasing.fit(data_aif)

# Aplica el modelo sin sesgo para predicciones
data_aif_pred = adv_debiasing.predict(data_aif)

# Fairness para el modelo original
metric_orig = BinaryLabelDatasetMetric(data_aif,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
print("Original dataset metrics:")
print(f"Difference in mean outcomes between unprivileged and privileged groups: {metric_orig.mean_difference():.6f}")

# Fairness para el Adversarial Debiasing
metric_pred = BinaryLabelDatasetMetric(data_aif_pred,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
print("\nAdversarial Debiasing dataset metrics:")
print(f"Difference in mean outcomes between unprivileged and privileged groups: {metric_pred.mean_difference():.6f}")

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 1.615634; batch adversarial loss: 0.679285
epoch 1; iter: 0; batch classifier loss: 0.770614; batch adversarial loss: 0.674977
epoch 2; iter: 0; batch classifier loss: 0.411477; batch adversarial loss: 0.709772
epoch 3; iter: 0; batch classifier loss: 0.306289; batch adversarial loss: 0.682934
epoch 4; iter: 0; batch classifier loss: 0.244092; batch adversarial loss: 0.693558
epoch 5; iter: 0; batch classifier loss: 0.175979; batch adversarial loss: 0.689615
epoch 6; iter: 0; batch classifier loss: 0.314036; batch adversarial loss: 0.704394
epoch 7; iter: 0; batch classifier loss: 0.273176; batch adversarial loss: 0.698767
epoch 8; iter: 0; batch classifier loss: 0.304900; batch adversarial loss: 0.699427
epoch 9; iter: 0; batch classifier loss: 0.264467; batch adversarial loss: 0.696887
epoch 10; iter: 0; batch classifier loss: 0.206590; batch adversarial loss: 0.702454
epoch 11; iter: 0; batch classifier loss: 0.260238; batch adversarial loss:

### 2.4.3) Post-procesamiento

In [ ]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

# Entrenamiento de un clasificador basado en el modelo original
scale = StandardScaler()
X_train = scale.fit_transform(data_aif.features)
y_train = data_aif.labels.ravel()
clf = LogisticRegression(random_state=66, max_iter=1000)
clf.fit(X_train, y_train)

# Las predicciones de los modelos
y_train_pred_prob = clf.predict_proba(X_train)[:, 1]  # Get probabilities for positive class

data_aif_pred = data_aif.copy(deepcopy=True)
data_aif_pred.scores = y_train_pred_prob.reshape(-1, 1)

eop = EqOddsPostprocessing(privileged_groups=privileged_groups,
                            unprivileged_groups=unprivileged_groups)
eop = eop.fit(data_aif, data_aif_pred)
data_aif_pred_eop = eop.predict(data_aif_pred)

# Fairness para el dataset original
metric_orig = BinaryLabelDatasetMetric(data_aif,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
print("Original dataset metrics:")
print(f"Difference in mean outcomes between unprivileged and privileged groups: {metric_orig.mean_difference():.6f}")

# Fairness para el dataset luego del Equalized Odds Post-Processing
metric_pred = BinaryLabelDatasetMetric(data_aif_pred_eop,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
print("\nEqualized Odds Post-Processing dataset metrics:")
print(f"Difference in mean outcomes between unprivileged and privileged groups: {metric_pred.mean_difference():.6f}")


Original dataset metrics:
Difference in mean outcomes between unprivileged and privileged groups: -0.002918

Equalized Odds Post-Processing dataset metrics:
Difference in mean outcomes between unprivileged and privileged groups: -0.002918


In [ ]:
from aif360.metrics import ClassificationMetric

# Métrica de clasificiación para el modelo luego del Equalized Odds Post-Processing
cm_pred = ClassificationMetric(data_aif, data_aif_pred_eop,
                               unprivileged_groups=unprivileged_groups,
                               privileged_groups=privileged_groups)


print(f"Difference in GFPR between unprivileged and privileged groups: {cm_pred.difference(cm_pred.generalized_false_positive_rate)}")
print(f"Difference in GFNR between unprivileged and privileged groups: {cm_pred.difference(cm_pred.generalized_false_negative_rate)}")

Difference in GFPR between unprivileged and privileged groups: -0.03325325549875968
Difference in GFNR between unprivileged and privileged groups: -0.008104288716572747


## 2.5) Análisis de Resultados


In [ ]:
# predicciones 1
precision_orig = precision_score(y_true, y_pred_lr)
print(f"Precision (Original Dataset): {precision_orig:.4f}")

# Predicciones del modelo rebalanceado
y_pred_rw = clf_rw.predict(X_test)
precision_rw = precision_score(y_true, y_pred_rw)
print(f"Precision (Reweighted Dataset): {precision_rw:.4f}")

Precision (Original Dataset): 0.9863
Precision (Reweighted Dataset): 0.9863


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [ ]:
# Predicciones 2
data_test_aif = BinaryLabelDataset(df=pd.concat([X_test, Y_test], axis=1),
                                   label_names=['approved'],
                                   protected_attribute_names=['mother_job', 'gender'])

# Predicciones del modelo adversario
data_test_pred_adv = adv_debiasing.predict(data_test_aif)
y_pred_adv = data_test_pred_adv.labels.ravel()
precision_adv = precision_score(Y_test, y_pred_adv)

# Imprime los valores de precisión
print(f"Precision (Original Model): {precision_orig:.4f}")
print(f"Precision (Adversarial Debiasing Model): {precision_adv:.4f}")

Precision (Original Model): 0.9863
Precision (Adversarial Debiasing Model): 0.9816


In [ ]:
# Predicciones 3
y_train_pred_eop = (data_aif_pred_eop.scores >= 0.5).astype(int)
precision_eop = precision_score(y_train, y_train_pred_eop)
print(f"Precision (Original Model): {precision_orig:.4f}")
print(f"Precision (Equalized Odds Post-Processing Dataset): {precision_eop:.4f}")

Precision (Original Model): 0.9863
Precision (Equalized Odds Post-Processing Dataset): 0.9899


## 2.6) Selección del Modelo Final


Al analizar los resultados podemos verificar que el modelo que mejor mitiga los sesgos sin afectar la precisión es el Reweighing en pre procesamiento, comparándolo con los otros modelos estos no logran cambiar los sesgos originales, incluso el Adversarial Debiasing traspasa el sesgo del grupo no privilegiado al privilegiado, produciendo sobrecompensación en la corrección.

Por lo tanto la estrategia de mitigación recomendada para este data set, es Reweighing en pre procesamiento.
